In [1]:
import numpy as np
from PIL import Image
import ctypes
import matplotlib.pyplot as plt
import random
import os

In [2]:
directory_inputs = ["../database/16x16/asie_sud_est","../database/16x16/rome_grece","../database/16x16/chateau_europe"]
directory_test_inputs = ["../database/16x16/tests/asie_sud_est","../database/16x16/tests/rome_grece","../database/16x16/tests/chateau_europe"]
# directory_inputs = ["../database/resized_dataset/amerique_sud","../database/resized_dataset/asie_sud_est","../database/resized_dataset/rome_grece"]

In [3]:
X = []
Y = []

for i in range(len(directory_inputs)):
    for image_file in os.listdir(directory_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([1., -1., -1.])
            elif i == 1 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1., 1., -1.])
            elif i == 2 :
                image_file = Image.open(directory_inputs[i]+'/'+image_file)
                # print(image_file)
                X.append(np.asarray(image_file))
                Y.append([-1.,-1.,1.])

X = np.array(X, dtype=ctypes.c_float)
X_flat = X.flatten()/255 * 2 - 1

Y = np.array(Y, dtype=ctypes.c_float)
Y_flat = Y.flatten()

In [4]:
len(X)

76775

In [5]:
X_test = []
Y_test = []

for i in range(len(directory_test_inputs)):
    for image_file in os.listdir(directory_test_inputs[i]):
        if 'image' in image_file:
            if i == 0 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_test.append(np.asarray(image_file))
                Y_test.append([1., -1.,-1.])
            elif i == 1 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_test.append(np.asarray(image_file))
                Y_test.append([-1., 1.,-1.])
            elif i == 2 :
                image_file = Image.open(directory_test_inputs[i]+'/'+image_file)
                # print(image_file)
                X_test.append(np.asarray(image_file))
                Y_test.append([-1., -1.,1.])

X_test = np.array(X_test, dtype=ctypes.c_float)
X_test_flat = X_test.flatten()/255 * 2 - 1

Y_test = np.array(Y_test, dtype=ctypes.c_float)
Y_test_flat = Y_test.flatten()

In [6]:
print(len(X_test))

19135


In [7]:
my_lib = ctypes.CDLL(r"..\rust_lib\target\release\rust_lib.dll")

my_lib.delete_float_array.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
]

my_lib.delete_float_array.restype = None

In [8]:
LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_train = (ctypes.c_float * len(X_flat))(*X_flat)
x_to_train_ptr = ctypes.cast(x_to_train, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_train = (ctypes.c_float * len(Y_flat))(*Y_flat)
y_to_train_ptr = ctypes.cast(y_to_train, LP_c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_test = (ctypes.c_float * len(X_test_flat))(*X_test_flat)
x_to_predict_test_ptr = ctypes.cast(x_to_predict_test, LP_c_float)
# LP_c_float = ctypes.POINTER(ctypes.c_float)
# y_to_predict_test = (ctypes.c_float * len(Y_train_flat))(*Y_train_flat)
# y_to_predict_test_ptr = ctypes.cast(y_to_predict_test, LP_c_float)

my_lib.radial_basis_function_model.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_float,
    ctypes.c_bool,
    ctypes.c_int32,
    ctypes.c_bool,
]

my_lib.radial_basis_function_model.restype = ctypes.POINTER(ctypes.c_float)

In [9]:
len(x_to_train)

58963200

In [ ]:
rbf_trained_model_ptr = my_lib.radial_basis_function_model(
    x_to_train_ptr,  # pointer_to_inputs_train : *mut f32,
    len(X),  # number_of_training_inputs : usize,
    x_to_predict_test_ptr,  # pointer_to_inputs_to_predict : *mut f32,
    len(X_test),  # number_of_inputs_to_predict : usize,
    len(X[0]),  # dimension_of_inputs : usize,
    y_to_train_ptr,  # pointer_to_labels : *mut f32,
    3,  # number_of_classes : usize,
    1,  # gamma : f32,
    True,  # is_classification : bool,
    9000,  # number_of_clusters : usize
    False, # is naif
)

rbf_trained_model = np.ctypeslib.as_array(rbf_trained_model_ptr, (len(X_test) * 3,))

print(len(rbf_trained_model))

In [ ]:
print(rbf_trained_model[0:30])

In [11]:
image_a_predire_grec = np.asarray(Image.open('test_images/greek/predict_grec_image.jpg').resize((16,16)).convert('RGB'))
image_a_predire_asie = np.asarray(Image.open('test_images/asie/predict_asie_image.jpg').resize((16,16)).convert('RGB'))
image_a_predire_chateau = np.asarray(Image.open('test_images/chateau/predict_chateau_image.jpg').resize((16,16)).convert('RGB'))

image_a_predire_data = np.array([image_a_predire_asie,image_a_predire_grec,image_a_predire_chateau],dtype=ctypes.c_float)
image_a_predire_data_flat=image_a_predire_data.flatten()



In [12]:
for j in range(0,len(image_a_predire_data_flat)-2,3) :
    if image_a_predire_data_flat[j] > image_a_predire_data_flat[j+1] and image_a_predire_data_flat[j] > image_a_predire_data_flat[j+2] :
        print("L'image a deviner vient asie")
    elif image_a_predire_data_flat[j+1] > image_a_predire_data_flat[j] and image_a_predire_data_flat[j+1] > image_a_predire_data_flat[j+2] :
        print("L'image a deviner vient rome")
    elif image_a_predire_data_flat[j+2] > image_a_predire_data_flat[j] and image_a_predire_data_flat[j+2] > image_a_predire_data_flat[j+1]:
        print("L'image a deviner vient de chateau")
    else :
        print('résultats non concluants')

L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient rome
L'image a deviner vient asie
L'image a deviner vient asie
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'image a deviner vient de chateau
L'i

In [ ]:
print(rbf_trained_model[0:15])